### Define functions and CNF logic

In [6]:
#install necessary packages
!pip install python-sat
from pysat.solvers import Minisat22
from itertools import combinations

In [2]:
#define necessary functions
def initialize_minesweeper(total_no_mines, no_cells_per_row, solver):
    #there are at least total_no_mines
    cell_indexes = list(range(1,no_cells_per_row**2+1))
    at_least_clause = [list(comb) for comb in combinations(cell_indexes, len(cell_indexes) - total_no_mines + 1)]
    for clause in at_least_clause:
        solver.add_clause(clause)
    #there are at most total_no_mines
    at_most_clause = [list(comb) for comb in combinations([-c for c in cell_indexes], total_no_mines+1)]
    for clause in at_most_clause:
        solver.add_clause(clause)
    return solver

def determine_cells_from_index(i, n):
    row = (i-1)//n
    column = (i-1)%n
    return (row, column)

def determine_adj_cells(i, n):
    adj_cells = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
    coordinates = determine_cells_from_index(i, n)
    if coordinates[0] == n-1:
        adj_cells = [cell for cell in adj_cells if cell[0] != 1]
    elif coordinates[0] == 0:
        adj_cells = [cell for cell in adj_cells if cell[0] != -1]
    
    if coordinates[1] == n-1:
        adj_cells = [cell for cell in adj_cells if cell[1] != 1]
    elif coordinates[1] == 0:
        adj_cells = [cell for cell in adj_cells if cell[1] != -1]
    return adj_cells
    
def add_clauses_for_m_mines(i, n, m, solver, no_mine_cells):
    adj_cells = determine_adj_cells(i, n)
    adj_cells_indexes = []
    for c in adj_cells:
        coord_n = determine_cells_from_index(i, n)
        index_i = (coord_n[0]+c[0])*n + (coord_n[1]+c[1]) + 1
        if index_i not in no_mine_cells:
            adj_cells_indexes.append(index_i)
    
    if m == 0:
        for cell in adj_cells_indexes:
            solver.add_clause([-cell])   
    else:
        # at least m mines
        at_least_clause = [list(comb) for comb in combinations(adj_cells_indexes, len(adj_cells_indexes)-m+1)]
        for clause in at_least_clause:
            solver.add_clause(clause)
        # at most m mines
        at_most_clause = [list(comb) for comb in combinations([-c for c in adj_cells_indexes], m+1)]
        for clause in at_most_clause:
            solver.add_clause(clause)
    return solver


def update_solver(no_mine_cells, open_cells_mines, discovered_mines, common_set, game_state, solver):
    if len(common_set) > 0:
        for i in common_set:
            x, y = determine_cells_from_index(abs(i), no_cells_per_row)
            if i > 0:
                game_state[x][y] = 'F'
                discovered_mines.append(abs(i))
            if i < 0:
                game_state[x][y] = solution[x][y]
                no_mine_cells.append(abs(i))
                no_mines = solution[x][y]
                open_cells_mines[abs(i)] = no_mines
    
    for cell in no_mine_cells:
        solver.add_clause([-cell])
    
    for cell in discovered_mines:
        solver.add_clause([cell])
        
    for cell, mines_no in open_cells_mines.items():
        solver = add_clauses_for_m_mines(cell, no_cells_per_row, mines_no, solver, no_mine_cells)
    
    return solver

def suggest_solution(solver, no_mine_cells, discovered_mines):
    models = list(solver.enum_models())
    hidden_cells_solutions = []
    common_set = []

    if len(models) == 1:
        hidden_cells_solutions = [i for i in models[0] if abs(i) not in no_mine_cells and abs(i) not in discovered_mines]
        common_set = hidden_cells_solutions
        
    elif len(models) > 1:
        for model in models:
            hidden_cells_solutions.append([i for i in model if abs(i) not in no_mine_cells and abs(i) not in discovered_mines])
        common_set = list(set.intersection(*map(set, hidden_cells_solutions)))
    
    return common_set, models

## Mode 1

In [3]:
#define main game
def play_minesweeper(total_no_mines, no_cells_per_row, solution):
    no_mine_cells = []
    discovered_mines = []
    open_cells_mines = {}
    common_set = []
    game_state = [['H']*no_cells_per_row for _ in range(no_cells_per_row)]
    
    #first step
    print("Welcome to Minesweeper! To start the game, please select the first cell from 1 to {}:".format(no_cells_per_row**2))
    index_input = input()
    x, y = determine_cells_from_index(int(index_input), no_cells_per_row)
    if solution[x][y] == 'M':
        print("\nYou opened a mine! You lost")
        return
    common_set.append(-int(index_input))
    
    while any('H' in row for row in game_state):
        Minesweeper = Minisat22()
        Minesweeper = initialize_minesweeper(total_no_mines, no_cells_per_row, Minesweeper)
        Minesweeper = update_solver(no_mine_cells, open_cells_mines, discovered_mines, common_set, game_state, Minesweeper)                 
        
        print("\nCurrent game state:")
        for row in game_state:
            print(*row)
        
        common_set, models = suggest_solution(Minesweeper, no_mine_cells, discovered_mines)
        if len(common_set)>0:
            posit_set = [i for i in common_set if i > 0]
            negat_set = [abs(i) for i in common_set if i < 0]
            if len(negat_set)>0:
                print("\nI think these cells are safe:", negat_set)
            if len(posit_set)>0:
                if len(negat_set)>0:
                    print("I think these cells have mines:", posit_set)
                else:
                    print("\nI think these cells have mines:", posit_set)
        else:
            model_count = {}
            for model in models:
                for atom in model:
                    if abs(atom) not in no_mine_cells and atom < 0:
                        if atom not in model_count:
                            model_count[atom] = 1/len(models)
                        else:
                            model_count[atom] += 1/len(models)
            if model_count:
                common_set = [max(model_count, key=model_count.get)]
                print("\nThere's a high possibility that this cell doesn't have a mine", abs(common_set[0]))
                x, y = determine_cells_from_index(abs(common_set[0]), no_cells_per_row)
                if solution[x][y] == 'M':
                    print("\nYou opened a mine! You lost")
                    print("\nThe solution was:")
                    for row in solution:
                        print(*row)
                    return
            else:
                print("\nNo further moves can be suggested.")
                break
    print("\nGame solved!")
    print("\nThe solution was:")
    for row in solution:
        print(*row)
    return Minesweeper

### Try Minesweeper on predefined solution

In [4]:
#initiate game

total_no_mines = 2
no_cells_per_row = 4
solution = [[1, 'M', 2, 1],
            [1, 1, 2, 'M'],
            [0, 0, 1, 1],
            [0, 0, 0, 0]]

In [7]:
Minesweeper = play_minesweeper(total_no_mines, no_cells_per_row, solution)

Welcome to Minesweeper! To start the game, please select the first cell from 1 to 16:
1

Current game state:
1 H H H
H H H H
H H H H
H H H H

There's a high possibility that this cell doesn't have a mine 4

Current game state:
1 H H 1
H H H H
H H H H
H H H H

I think these cells are safe: [16, 15, 14, 13, 12, 11, 10, 9]

Current game state:
1 H H 1
H H H H
0 0 1 1
0 0 0 0

I think these cells are safe: [3, 5, 6, 7]
I think these cells have mines: [2, 8]

Current game state:
1 F 2 1
1 1 2 F
0 0 1 1
0 0 0 0

No further moves can be suggested.

Game solved!

The solution was:
1 M 2 1
1 1 2 M
0 0 1 1
0 0 0 0


### Try Minesweeper on generated game

In [8]:
import random

def generate_minesweeper(n_field, n_mines):
    #generate empty field of n_field x n_field with zeros
    field = [[0]*n_field for i in range(n_field)]
    
    #randomly add n_mines of mines into the field
    m = 0
    while m < n_mines:
        r = random.randint(0, n_field-1)
        c = random.randint(0, n_field-1)
        if field[r][c] != 'M':
            field[r][c] = 'M'
            m += 1
    #calculate number of mines around each position
    neighbors = [(-1, 1), (0, 1), (1, 1), 
                 (-1, 0),         (1, 0), 
                 (-1,-1), (0,-1), (1,-1)]
    
    for i in range(n_field):
        for j in range(n_field):
            if field[i][j] != 'M':
                for dx, dy in neighbors:
                    ni, nj = i + dx, j + dy
                    if 0 <= ni < n_field and 0 <= nj < n_field and field[ni][nj] == 'M':
                        field[i][j] += 1
                
    return field

In [9]:
total_no_mines = 3
no_cells_per_row = 6
solution = generate_minesweeper(no_cells_per_row, total_no_mines)

In [10]:
Minesweeper = play_minesweeper(total_no_mines, no_cells_per_row, solution)

Welcome to Minesweeper! To start the game, please select the first cell from 1 to 36:
3

Current game state:
H H 1 H H H
H H H H H H
H H H H H H
H H H H H H
H H H H H H
H H H H H H

There's a high possibility that this cell doesn't have a mine 6

Current game state:
H H 1 H H 0
H H H H H H
H H H H H H
H H H H H H
H H H H H H
H H H H H H

I think these cells are safe: [5, 12, 11]

Current game state:
H H 1 H 0 0
H H H H 1 1
H H H H H H
H H H H H H
H H H H H H
H H H H H H

I think these cells are safe: [16, 4, 10]

Current game state:
H H 1 1 0 0
H H H 2 1 1
H H H 2 H H
H H H H H H
H H H H H H
H H H H H H

I think these cells are safe: [22, 21, 23, 8, 2]
I think these cells have mines: [9]

Current game state:
H 1 1 1 0 0
H 1 F 2 1 1
H H H 2 H H
H H 0 1 1 H
H H H H H H
H H H H H H

I think these cells are safe: [30, 29, 28, 27, 26, 24, 20, 18, 15, 14, 13, 7, 1]
I think these cells have mines: [17]

Current game state:
0 1 1 1 0 0
0 1 F 2 1 1
0 1 1 2 F 1
H 0 0 1 1 1
H 0 0 0 1 1
H H H H H 

## Mode 2

In [11]:
#install pygame
!pip install pygame

In [12]:
#get location of pygame and import pygame
import sys
output = !pip show pygame
for line in output:
    if line.startswith("Location: "):
        location = line.split(":",1)[1].strip()
        break
sys.path.insert(1, location)
import pygame

pygame 2.6.0 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
#load images
tile_hidden = pygame.image.load("Grid.png")
tile_empty = pygame.image.load("empty.png")
tile_flag = pygame.image.load("flag.png")
tile_1 = pygame.image.load("grid1.png")
tile_2 = pygame.image.load("grid2.png")
tile_3 = pygame.image.load("grid3.png")
tile_4 = pygame.image.load("grid4.png")
tile_5 = pygame.image.load("grid5.png")
tile_6 = pygame.image.load("grid6.png")
tile_7 = pygame.image.load("grid7.png")
tile_8 = pygame.image.load("grid8.png")
tile_mine = pygame.image.load("mine.png")
tile_mine_selected = pygame.image.load("mineClicked.png")
tile_mine_false = pygame.image.load("mineFalse.png")
safe_tile = pygame.image.load("safe_cell.png")
detect_tile = pygame.image.load("detected_mine.png")
unsure_tile = pygame.image.load("unsure_cell.png")

In [14]:
#rewrite main minesweeper function to give hints
def hint_minesweeper(total_no_mines, no_cells_per_row, solution, current_state):
    no_mine_cells = []
    common_set = []
    discovered_mines = []
    open_cells_mines = {}
    
    for i in range(no_cells_per_row):
        for j in range(no_cells_per_row):
            if current_state[i][j] != "H":
                index = i*no_cells_per_row + j + 1
                no_mine_cells.append(index)
                open_cells_mines[index] = solution[i][j]
    
    Minesweeper = Minisat22()
    Minesweeper = initialize_minesweeper(total_no_mines, no_cells_per_row, Minesweeper)
    Minesweeper = update_solver(no_mine_cells, open_cells_mines, discovered_mines, common_set, current_state, Minesweeper)                 
    
    common_set, models = suggest_solution(Minesweeper, no_mine_cells, discovered_mines)    
    return common_set, models

In [15]:
#define game parameters here:
no_cells_per_row = 6
total_no_mines = 3

### Run the cell below to play the game

In [16]:
#initiate pygame and define processes
pygame.init()

#initiate game mode, other modes are "won", 'lost'
game_mode = "playing"

#display size and borders
tile_size = 32  # size of one tile
border = 64  # left, bottom border
large_border = 96  # top border, bottom border is 2*large_border
display_width = tile_size * no_cells_per_row + 2*border  # Display width
display_height = tile_size * no_cells_per_row + 3*large_border - 36  # Display height
gameDisplay = pygame.display.set_mode((display_width, display_height))  # Create display

#colors
text_pink = (255, 192, 203)
button_pink = (255, 128, 150)

#fonts
pygame.font.init()
smallfont = pygame.font.SysFont('Corbel',22)
smallerfont = pygame.font.SysFont('Corbel',18) 

#button dimensions
button_width = 128
button_height = 36
button_x = display_width/2 - button_width/2
button_y = display_height - 2*large_border + 64
main_button = pygame.Surface((button_width, button_height))

#define function to initiate a game
def initiate_game(no_cells_per_row, total_no_mines):
    global game_mode
    game_mode = 'playing'
    solution = generate_minesweeper(no_cells_per_row, total_no_mines)
    for i in range(no_cells_per_row):
        for j in range(no_cells_per_row):
            gameDisplay.blit(tile_hidden, (border+tile_size*j, large_border+tile_size*i))
    current_state = [['H']*no_cells_per_row for _ in range(no_cells_per_row)]
    #texts
    pygame.draw.rect(gameDisplay, (0,0,0), pygame.Rect(0, 0, display_width, large_border))
    pygame.draw.rect(gameDisplay, (0,0,0), pygame.Rect(0, button_y + button_height, display_width, display_height-(button_y + button_height)))
    welcome_text_1 = smallfont.render('Welcome to Minesweeper!', True, text_pink)
    welcome_text_rect_1 = welcome_text_1.get_rect(center=(display_width/2, 36))
    gameDisplay.blit(welcome_text_1, welcome_text_rect_1)
    welcome_text_2 = smallfont.render('Select a cell to open', True, text_pink)
    welcome_text_rect_2 = welcome_text_2.get_rect(center=(display_width/2, 64))
    gameDisplay.blit(welcome_text_2, welcome_text_rect_2)
    #button
    main_button.fill(button_pink)
    button_text = smallerfont.render('I need advice!', True, (255, 255, 255))
    main_button.blit(button_text, (12, 9))
    gameDisplay.blit(main_button, (button_x, button_y))  
    return solution, current_state

#define a function to open a cell and change the tile
def open_cell(solution, i, j):
    global game_mode
    global current_state
    if solution[i][j] == 1:
        gameDisplay.blit(tile_1, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 2:
        gameDisplay.blit(tile_2, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 3:
        gameDisplay.blit(tile_3, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 4:
        gameDisplay.blit(tile_4, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 5:
        gameDisplay.blit(tile_5, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 6:
        gameDisplay.blit(tile_6, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 7:
        gameDisplay.blit(tile_7, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 8:
        gameDisplay.blit(tile_8, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    elif solution[i][j] == 0:
        gameDisplay.blit(tile_empty, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
    
    elif solution[i][j] == "M" and game_mode == 'playing':
        #what happens when player loses
        game_mode = "lost"
        gameDisplay.blit(tile_mine_selected, (border+tile_size*j, large_border+tile_size*i))
        current_state[i][j] = solution[i][j]
        #open other mines
        for k in range(len(current_state)):
            for n in range(len(current_state[k])):
                if current_state[k][n] == 'H' and solution[k][n] == 'M':
                    gameDisplay.blit(tile_mine, (border+tile_size*n, large_border+tile_size*k))
        #text change
        pygame.draw.rect(gameDisplay, (0,0,0), pygame.Rect(0, 0, display_width, large_border))
        losing_text_1 = smallfont.render('Oh no, you opened a mine!', True, text_pink)
        losing_text_rect_1 = losing_text_1.get_rect(center=(display_width/2, 36))
        gameDisplay.blit(losing_text_1, losing_text_rect_1)
        losing_text_2 = smallfont.render('Please try again', True, text_pink)
        losing_text_rect_2 = losing_text_2.get_rect(center=(display_width/2, 64))
        gameDisplay.blit(losing_text_2, losing_text_rect_2)
        #button change
        main_button.fill(button_pink)
        try_button_text = smallerfont.render('Try again', True, (255, 255, 255))
        main_button.blit(try_button_text, (32, 9))
        gameDisplay.blit(main_button, (button_x, button_y))
    
    #what happens when player wins
    if game_mode == 'playing' and all(current_state[i][j] != 'H' or solution[i][j] == 'M' for i in range(len(current_state)) for j in range(len(current_state[i]))):
        game_mode = 'won'
        #text
        pygame.draw.rect(gameDisplay, (0,0,0), pygame.Rect(0, 0, display_width, large_border))
        pygame.draw.rect(gameDisplay, (0,0,0), pygame.Rect(0, button_y + button_height, display_width, display_height-(button_y + button_height)))
        winning_text_1 = smallfont.render('You won!', True, text_pink)
        winning_text_rect_1 = winning_text_1.get_rect(center=(display_width/2, 36))
        gameDisplay.blit(winning_text_1, winning_text_rect_1)
        winning_text_2 = smallfont.render('Would you like to play again?', True, text_pink)
        winning_text_rect_2 = winning_text_2.get_rect(center=(display_width/2, 64))
        gameDisplay.blit(winning_text_2, winning_text_rect_2)
        #button
        main_button.fill(button_pink)
        button_text = smallerfont.render('Play again', True, (255, 255, 255))
        main_button.blit(button_text, (24, 9))
        gameDisplay.blit(main_button, (button_x, button_y))
        #change all unopened mine tiles to flags
        for p in range(len(solution)):
            for q in range(len(solution[p])):
                if solution[p][q] == 'M':
                    gameDisplay.blit(tile_flag, (border+tile_size*q, large_border+tile_size*p))

def suggestion_from_model_counting(models):
    model_count = {}
    for model in models:
        for atom in model:
            x, y = determine_cells_from_index(abs(atom), no_cells_per_row)
            if current_state[x][y] == 'H' and atom < 0:
                if atom not in model_count:
                    model_count[atom] = 1/len(models)
                else:
                    model_count[atom] += 1/len(models)
    if model_count:
        high_prob = [max(model_count, key=model_count.get)]
        x, y = determine_cells_from_index(abs(high_prob[0]), no_cells_per_row)
        gameDisplay.blit(unsure_tile, (border+tile_size*y, large_border+tile_size*x))
        suggestion_text = smallerfont.render("I suggest to open the blue tile", True, text_pink)
        suggestion_text_rect = suggestion_text.get_rect(center=(display_width/2, button_y+button_height+32))
        gameDisplay.blit(suggestion_text, suggestion_text_rect)

                    
#window caption
pygame.display.set_caption('Minesweeper')

#initiate game
solution, current_state = initiate_game(no_cells_per_row, total_no_mines)     
pygame.display.flip()

#running = while game is running
running = True
while running: 
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            pos = pygame.mouse.get_pos()
            #if main button is pressed
            if button_x <= pos[0] <= button_x + button_width and button_y <= pos[1] <= button_y + button_height:
                if game_mode == 'playing':
                    
                    if all(current_state[i][j] == 'H' for i in range(len(current_state)) for j in range(len(current_state[i]))):
                        no_solution_text_1 = smallerfont.render('No suggestions yet!', True, text_pink)
                        no_solution_text_rect_1 = no_solution_text_1.get_rect(center=(display_width/2, button_y+button_height+32))
                        gameDisplay.blit(no_solution_text_1, no_solution_text_rect_1)
                        no_solution_text_2 = smallerfont.render('Please open a cell first', True, text_pink)
                        no_solution_text_rect_2 = no_solution_text_2.get_rect(center=(display_width/2, button_y+button_height+56))
                        gameDisplay.blit(no_solution_text_2, no_solution_text_rect_2)
                    
                    else:
                        #give hint
                        pygame.draw.rect(gameDisplay, (0,0,0), pygame.Rect(0, button_y + button_height, display_width, display_height-(button_y + button_height)))
                        text_thinking = smallfont.render('Thinking...', True, text_pink)
                        text_thinking_rect = text_thinking.get_rect(center=(display_width/2, button_y+button_height+32))
                        gameDisplay.blit(text_thinking, text_thinking_rect)
                        pygame.display.update()
                    
                        common_set, models = hint_minesweeper(total_no_mines, no_cells_per_row, solution, current_state)
                        pygame.draw.rect(gameDisplay, (0,0,0), pygame.Rect(0, button_y + button_height, display_width, display_height-(button_y + button_height)))
                    
                        if len(common_set)>0:
                            posit_set = [i for i in common_set if i > 0]
                            negat_set = [abs(i) for i in common_set if i < 0]
                            for pos_index in posit_set:
                                x, y = determine_cells_from_index(pos_index, no_cells_per_row)
                                gameDisplay.blit(detect_tile, (border+tile_size*y, large_border+tile_size*x))
                            posit_text = smallerfont.render('I think mines are under pink tiles.', True, text_pink)
                            posit_text_rect = posit_text.get_rect(center=(display_width/2, button_y+button_height+56))
                            gameDisplay.blit(posit_text, posit_text_rect)
                            
                            if len(negat_set)>0:
                                for neg_index in negat_set:
                                    x, y = determine_cells_from_index(neg_index, no_cells_per_row)
                                    gameDisplay.blit(safe_tile, (border+tile_size*y, large_border+tile_size*x))
                                                    
                                negat_text = smallerfont.render('Safe tiles are colored green', True, text_pink)
                                negat_text_rect = negat_text.get_rect(center=(display_width/2, button_y+button_height+32))
                                gameDisplay.blit(negat_text, negat_text_rect)
                            elif len(negat_set) == 0:
                                suggestion_from_model_counting(models) 
                        else:
                            suggestion_from_model_counting(models)  
                        
                    pygame.display.update()
                
                #initiate the game again if player wins or looses instead
                elif game_mode == 'lost' or game_mode == 'won':
                    solution, current_state = initiate_game(no_cells_per_row, total_no_mines)
            
            #if player presses a tile, open it        
            elif border <= pos[0] <= border+tile_size * no_cells_per_row and large_border <= pos[1] <= large_border+tile_size * no_cells_per_row:
                x = (pos[0] - border)//tile_size
                y = (pos[1] - large_border)//tile_size
                open_cell(solution, y, x)
                
    pygame.display.update()        
pygame.quit()
sys.exit()

SystemExit: 

C:\Users\User\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
